# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 28 2023 8:30AM,258320,10,0086456167,ISDIN Corporation,Released
1,Feb 28 2023 8:30AM,258320,10,0086456169,ISDIN Corporation,Released
2,Feb 28 2023 8:30AM,258320,10,0086457010,ISDIN Corporation,Released
3,Feb 28 2023 8:30AM,258320,10,0086457011,ISDIN Corporation,Released
4,Feb 28 2023 8:30AM,258322,10,0086453318,ISDIN Corporation,Released
5,Feb 28 2023 8:30AM,258322,10,0086455751,ISDIN Corporation,Released
6,Feb 28 2023 8:30AM,258320,10,0086455331,ISDIN Corporation,Released
7,Feb 28 2023 8:30AM,258320,10,0086455333,ISDIN Corporation,Released
8,Feb 28 2023 8:30AM,258320,10,0086455328,ISDIN Corporation,Released
9,Feb 28 2023 8:30AM,258320,10,0086455329,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,258314,Released,1
35,258317,Released,2
36,258319,Released,12
37,258320,Released,37
38,258322,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258314,NaN,NaN,1.0
258317,NaN,NaN,2.0
258319,NaN,NaN,12.0
258320,NaN,NaN,37.0
258322,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258162,0.0,9.0,48.0
258166,16.0,1.0,0.0
258188,19.0,5.0,0.0
258201,0.0,0.0,1.0
258202,87.0,11.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258162,0,9,48
258166,16,1,0
258188,19,5,0
258201,0,0,1
258202,87,11,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258162,0,9,48
1,258166,16,1,0
2,258188,19,5,0
3,258201,0,0,1
4,258202,87,11,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258162,,9,48
1,258166,16,1,
2,258188,19,5,
3,258201,,,1
4,258202,87,11,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 28 2023 8:30AM,258320,10,ISDIN Corporation
4,Feb 28 2023 8:30AM,258322,10,ISDIN Corporation
36,Feb 28 2023 8:30AM,258307,10,ISDIN Corporation
72,Feb 28 2023 8:30AM,258302,10,ISDIN Corporation
96,Feb 28 2023 8:30AM,258296,10,ISDIN Corporation
150,Feb 28 2023 8:30AM,258291,10,ISDIN Corporation
198,Feb 28 2023 8:30AM,258317,10,"Methapharm, Inc."
200,Feb 28 2023 8:30AM,258319,10,"Methapharm, Inc."
216,Feb 27 2023 4:23PM,258314,19,"Avet Pharmaceuticals Labs, Inc"
217,Feb 27 2023 4:16PM,258310,19,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 28 2023 8:30AM,258320,10,ISDIN Corporation,,,37
1,Feb 28 2023 8:30AM,258322,10,ISDIN Corporation,,,2
2,Feb 28 2023 8:30AM,258307,10,ISDIN Corporation,,,38
3,Feb 28 2023 8:30AM,258302,10,ISDIN Corporation,,,21
4,Feb 28 2023 8:30AM,258296,10,ISDIN Corporation,,,57
5,Feb 28 2023 8:30AM,258291,10,ISDIN Corporation,,,47
6,Feb 28 2023 8:30AM,258317,10,"Methapharm, Inc.",,,2
7,Feb 28 2023 8:30AM,258319,10,"Methapharm, Inc.",,,12
8,Feb 27 2023 4:23PM,258314,19,"Avet Pharmaceuticals Labs, Inc",,,1
9,Feb 27 2023 4:16PM,258310,19,"Methapharm, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 8:30AM,258320,10,ISDIN Corporation,37,,
1,Feb 28 2023 8:30AM,258322,10,ISDIN Corporation,2,,
2,Feb 28 2023 8:30AM,258307,10,ISDIN Corporation,38,,
3,Feb 28 2023 8:30AM,258302,10,ISDIN Corporation,21,,
4,Feb 28 2023 8:30AM,258296,10,ISDIN Corporation,57,,
5,Feb 28 2023 8:30AM,258291,10,ISDIN Corporation,47,,
6,Feb 28 2023 8:30AM,258317,10,"Methapharm, Inc.",2,,
7,Feb 28 2023 8:30AM,258319,10,"Methapharm, Inc.",12,,
8,Feb 27 2023 4:23PM,258314,19,"Avet Pharmaceuticals Labs, Inc",1,,
9,Feb 27 2023 4:16PM,258310,19,"Methapharm, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 8:30AM,258320,10,ISDIN Corporation,37,,
1,Feb 28 2023 8:30AM,258322,10,ISDIN Corporation,2,,
2,Feb 28 2023 8:30AM,258307,10,ISDIN Corporation,38,,
3,Feb 28 2023 8:30AM,258302,10,ISDIN Corporation,21,,
4,Feb 28 2023 8:30AM,258296,10,ISDIN Corporation,57,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 8:30AM,258320,10,ISDIN Corporation,37.0,NaN,NaN
1,Feb 28 2023 8:30AM,258322,10,ISDIN Corporation,2.0,NaN,NaN
2,Feb 28 2023 8:30AM,258307,10,ISDIN Corporation,38.0,NaN,NaN
3,Feb 28 2023 8:30AM,258302,10,ISDIN Corporation,21.0,NaN,NaN
4,Feb 28 2023 8:30AM,258296,10,ISDIN Corporation,57.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 28 2023 8:30AM,258320,10,ISDIN Corporation,37.0,0.0,0.0
1,Feb 28 2023 8:30AM,258322,10,ISDIN Corporation,2.0,0.0,0.0
2,Feb 28 2023 8:30AM,258307,10,ISDIN Corporation,38.0,0.0,0.0
3,Feb 28 2023 8:30AM,258302,10,ISDIN Corporation,21.0,0.0,0.0
4,Feb 28 2023 8:30AM,258296,10,ISDIN Corporation,57.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4648815,405.0,14.0,20.0
12,258245,0.0,1.0,0.0
15,258202,15.0,11.0,87.0
16,774765,2.0,1.0,0.0
18,258279,1.0,0.0,0.0
19,1033071,23.0,1.0,16.0
20,258284,21.0,2.0,0.0
21,258234,1.0,0.0,0.0
22,516554,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4648815,405.0,14.0,20.0
1,12,258245,0.0,1.0,0.0
2,15,258202,15.0,11.0,87.0
3,16,774765,2.0,1.0,0.0
4,18,258279,1.0,0.0,0.0
5,19,1033071,23.0,1.0,16.0
6,20,258284,21.0,2.0,0.0
7,21,258234,1.0,0.0,0.0
8,22,516554,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,405.0,14.0,20.0
1,12,0.0,1.0,0.0
2,15,15.0,11.0,87.0
3,16,2.0,1.0,0.0
4,18,1.0,0.0,0.0
5,19,23.0,1.0,16.0
6,20,21.0,2.0,0.0
7,21,1.0,0.0,0.0
8,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,405.0
1,12,Released,0.0
2,15,Released,15.0
3,16,Released,2.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,20.0,0.0,87.0,0.0,0.0,16.0,0.0,0.0,0.0
Executing,14.0,1.0,11.0,1.0,0.0,1.0,2.0,0.0,0.0
Released,405.0,0.0,15.0,2.0,1.0,23.0,21.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21,22
0,Completed,20.0,0.0,87.0,0.0,0.0,16.0,0.0,0.0,0.0
1,Executing,14.0,1.0,11.0,1.0,0.0,1.0,2.0,0.0,0.0
2,Released,405.0,0.0,15.0,2.0,1.0,23.0,21.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21,22
0,Completed,20.0,0.0,87.0,0.0,0.0,16.0,0.0,0.0,0.0
1,Executing,14.0,1.0,11.0,1.0,0.0,1.0,2.0,0.0,0.0
2,Released,405.0,0.0,15.0,2.0,1.0,23.0,21.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()